In [1]:
import json
from collections import deque

# Collect Data

Get data from https://lostcircles.com/. Download JSON(no pics) after up finish loading your network. Run the following function to get a adjacency list. One thing about the json file is that some edges only have one directions recorded but we should add both directions to the adj list.

In [2]:
path = 'fb_network.json'

def process_json(path): # Parse the json file into adjacency list and node list
    data = json.load(open(path))
    links = data["links"]
    nodes = data["nodes"]
    adj_list = {}
    for edge in links:
        
        if edge["source"] in adj_list:
            if edge["target"] not in adj_list[edge["source"]]:
                adj_list[edge["source"]].append(edge["target"])
        else:
            adj_list[edge["source"]] = [edge["target"]]
            
        
        if edge["target"] in adj_list:
            if edge["source"] not in adj_list[edge["target"]]:
                adj_list[edge["target"]].append(edge["source"])
        else:
            adj_list[edge["target"]] = [edge["source"]]
            
    return adj_list, nodes

adj_list, nodes = process_json(path)

# Tie Strength

In [3]:
c = 0    ## You can look up the index using a user's name
for i in nodes:
    if i['name'] == 'Andrew Cui':
        print(c)
    c += 1

11


Then we try to implement all the math functions mentioned in the paper.

In [4]:
def common_neighbor(adj_list, node1, node2):  #Find the list of common neighbors of node1 and node2
    result = []
    for nei in adj_list[node1]:
        if nei in adj_list[node2]:
            result.append(nei)
    return result


def dispersion(adj_list, node1, node2, threshold = 1, normalized = False): #Calculate the dispersion
    common_nei = common_neighbor(adj_list, node1, node2)
    result = 0
    for i in range(len(common_nei) - 1):
        for j in range(i + 1, len(common_nei)):
            result += distance(threshold, adj_list, common_nei[i], common_nei[j]) #Distance function gives 1 or 0
    if normalized:
        if len(common_nei) <= 1:  # If the two nodes have less or equal to 1 mutual friends, we set the dispersion to 0
            return 0
        return result/len(common_nei)

    return result    # If not normalized, return the result directly

def distance(threshold, adj_list, u, v): 
    # Use BFS to check if distance between u and v are within threshold, return 1 when dist > threshold, 0 when <=
    
    # if len(common_neighbor(adj_list, u, v)) > 2:
    #   return 0 
    
    # The previous two lines: according to the paper, we should consider the distance to be 0 if the two nodes have
    # mutual friends other than the two nodes we are calculating dispersion of.
    
    # We are ignoring this rule in this activity since our own fb network has limited scale

    queue = deque([u]) # Queue data structure
    explored = {u} # Set data structure for O(1) lookup
    count = 0
    while(len(queue) != 0 and count < threshold):
        cur_layer = len(queue)
        # We record the number of nodes in the current layer, so we can keep track of how many layers we have explored
        # E.g. if the threshold is 3, we only want to explore if v is within 3 layers of u.
        for i in range(cur_layer):  
            # In the for loop, we explore all nodes in current layer.
            cur = queue.popleft()
            for nei in adj_list[cur]:
                if nei == v:
                    return 0
                if nei not in explored:
                    queue.append(nei)
                    explored.add(nei)
                    
        count += 1 
    return 1  # This means we did not find v within 3 layers of u, so the distance should be 1.

In [5]:
adj = {}   # This is the graph in the slide, used for testing
adj['a'] = ['b', 'c', 'u']
adj['b'] = ['a', 'c', 'd', 'e', 'f', 'u']
adj['c'] = ['a', 'b', 'd', 'f', 'h', 'u']
adj['d'] = ['c', 'b', 'f', 'u']
adj['e'] = ['b', 'f', 'u']
adj['f'] = ['c', 'd', 'b', 'e', 'h', 'u']
adj['g'] = ['u']
adj['h'] = ['c', 'f', 'k', 'j', 'u']
adj['i'] = ['k', 'j', 'u']
adj['j'] = ['h', 'u', 'k', 'i']
adj['k'] = ['i', 'j', 'u']
adj['u'] = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']

In [6]:
distance(1, adj, 'a', 'b')

0

'a' and 'b' are not directly connected.

In [7]:
dispersion(adj, 'h', 'u')

4

In [8]:
dispersion(adj, 'h', 'u', 1, True)

1.0

The dispersion and normalized dispersion between h and u are 4 and 1.

In [9]:
nodes[11]

{'dataUrl': '',
 'id': '100009517843209',
 'name': 'Andrew Cui',
 'profile': 'https://www.facebook.com/andrew.cui.75?fref=pb&hc_location=friends_tab',
 'userName': 'andrew.cui.75'}

In [10]:
nodes[30]

{'dataUrl': '',
 'id': '100004068944907',
 'name': 'Ayoub Belemlih',
 'profile': 'https://www.facebook.com/belemlih.ayoub?fref=pb&hc_location=friends_tab',
 'userName': 'belemlih.ayoub'}

In [11]:
dispersion(adj_list, 11, 30)

155

In [12]:
dispersion(adj_list, 11, 30, 1, True)

5.0

In [13]:
def recursive_dispersion(adj_list, node, max_iterations = 1, threshold = 1):
    values = {}  # dictionary with [nodeX : [1, dispersion after 1 recursion, dispersion after 2 recursions...]]
    for nei in adj_list.keys():   # Initialize all the dispersion values to be list with only 1
        if nei != node:
            values[nei] = [1]
    
    iteration_num = 0
    while iteration_num < max_iterations:   # Run a certain number of recursions
        for nei in values.keys():  
            values[nei].append(helper(adj_list, node, nei, values, iteration_num, threshold)) 
            # Use the equation and the previous round of dispersion values to calculate the new dispersion value for
            # node nei. Then append the new value to the list of nei.
        iteration_num += 1
        
    return values  # Return the dictionary


def helper(adj_list, u, v, values, iteration_num, threshold):  
    # This helper function is the math equation given in the paper
    common_nei = common_neighbor(adj_list, u, v)
    if len(common_nei) <= 1:
        return 0
    
    result = 0
    for nei in common_nei:
        result += values[nei][iteration_num] * values[nei][iteration_num]
    
    for i in range(len(common_nei) - 1):
        for j in range(i + 1, len(common_nei)):
            result += 2 * distance(threshold, adj_list, common_nei[i], common_nei[j]) * values[common_nei[i]][iteration_num] * values[common_nei[j]][iteration_num]
    
    return result/len(common_nei)


def find_max_dispersion(dispersions):
    # Find the key according to the largest dispersion value
    result = 0
    max_dispersion = 0
    for key in dispersions.keys():
        if dispersions[key][len(dispersions[key]) - 1] > max_dispersion:
            max_dispersion = dispersions[key][len(dispersions[key]) - 1] #Compare the last number in each list
            result = key
    return result

In [14]:
dispersions_test = recursive_dispersion(adj, 'u', 3,  1)

Test it on the graph in the slide: We use max_iterations = 3 since the paper says it gives the best result. Threshold is set to be 1 (directly connected). We look for the dispersion values between all other nodes and u.

In [15]:
dispersions_test

{'a': [1, 1.0, 9.0, 120.2],
 'b': [1, 3.0, 7.8, 314.104],
 'c': [1, 3.0, 13.4, 336.0172839506173],
 'd': [1, 1.0, 9.0, 139.98666666666668],
 'e': [1, 1.0, 9.0, 120.2],
 'f': [1, 3.0, 13.4, 336.0172839506173],
 'g': [1, 0, 0, 0],
 'h': [1, 3.0, 13.444444444444445, 199.94419753086422],
 'i': [1, 1.0, 1.888888888888889, 17.839506172839506],
 'j': [1, 1.6666666666666667, 5.666666666666667, 79.559670781893],
 'k': [1, 1.0, 1.888888888888889, 17.839506172839506]}

The last values in the lists are the results after 3 iterations.

In [16]:
find_max_dispersion(dispersions_test)

'c'

In [17]:
dispersions = recursive_dispersion(adj_list, 11, 3, 1)

Run the recursive dispersion function on your own fb network starting with one of your friend. I am using 11: Andrew Cui here. (You will not be in your own network)

In [18]:
dispersions

{0: [1, 17.13953488372093, 5167.573150509105, 512880296.06353384],
 1: [1, 0, 0, 0],
 2: [1, 19.325581395348838, 5053.959658260692, 534150213.11327785],
 3: [1, 13.235294117647058, 5224.906952643313, 478369513.5862104],
 4: [1, 22.59259259259259, 7256.642483296797, 750275451.3137587],
 5: [1, 1.6666666666666667, 805.6599938781759, 61046071.3248296],
 6: [1, 1.6666666666666667, 805.6599938781759, 61046071.3248296],
 7: [1, 1.0, 1148.622314049587, 84415023.31589775],
 8: [1, 29.545454545454547, 9335.620715263272, 938684753.3203942],
 9: [1, 12.090909090909092, 3946.381220025741, 366938805.79666495],
 10: [1, 1.6666666666666667, 805.6599938781759, 61046071.3248296],
 12: [1, 24.89090909090909, 7938.226199238651, 778159360.4452263],
 13: [1, 25.0, 8548.92248155026, 877129119.263357],
 14: [1, 4.428571428571429, 885.6377721204581, 93847020.21700935],
 15: [1, 8.538461538461538, 3333.8265510145916, 319110298.08345026],
 16: [1, 41.2, 10858.894467406419, 1121683216.724293],
 17: [1, 2.0, 359.

In [19]:
find_max_dispersion(dispersions)

45

In [20]:
nodes[45]

{'dataUrl': '',
 'id': '100001107732647',
 'name': 'Jongwon Han',
 'profile': 'https://www.facebook.com/hans.jongwon?fref=pb&hc_location=friends_tab',
 'userName': 'hans.jongwon'}